In [21]:
import sys
sys.path.append("../src")

In [22]:
from CurvyCUSIPs.CurveDataFetcher import CurveDataFetcher
from CurvyCUSIPs.utils.BBG490CurveUtils import BBG490CurveUtils
from CurvyCUSIPs.utils.dtcc_swaps_utils import tenor_to_years
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import QuantLib as ql
import plotly.graph_objects as go
import statsmodels.api as sm
from typing import Dict, List, Optional, Tuple
import os
from pandas.tseries.offsets import CustomBusinessDay 
from pandas.tseries.holiday import USFederalHolidayCalendar

from dotenv import dotenv_values
env_path = os.path.join(os.getcwd(), "../.env")
print(env_path)
config = dotenv_values(env_path)

import nest_asyncio
nest_asyncio.apply()

import warnings
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=pd.errors.SettingWithCopyWarning)

%load_ext autoreload
%autoreload 2

c:\Users\chris\Project Bond King\Curvy-CUSIPs\notebooks\../.env
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
curve_data_fetcher = CurveDataFetcher(use_ust_issue_date=True, fred_api_key=config["FRED_API_KEY"])

In [24]:
bbg_490_curve = BBG490CurveUtils(s490_curve_db_path=r"C:\Users\chris\Project Bond King\ql_swap_curve_objs\nyclose_s490_usd_sofr_curve_mixed_master")

# SOFR OIS NY Close Curve:

In [25]:
as_of_dates = [datetime(2024, 11, 27), datetime(2024, 11, 29)]

fig = go.Figure()
for curr in as_of_dates:
    try:
        str_ts = str(int(curr.timestamp()))
        ohlc_df = pd.DataFrame(bbg_490_curve.s490_nyclose_db.get(str_ts)["ohlc"])
        ohlc_df["Expiry"] = pd.to_numeric(ohlc_df["Expiry"], errors="coerce")
        ohlc_df["Expiry"] = pd.to_datetime(ohlc_df["Expiry"], errors="coerce", unit="s")
        if len(as_of_dates) == 1:
            display(ohlc_df)
        fig.add_trace(go.Scatter(x=((ohlc_df["Expiry"] - curr).dt.days / 360), y=ohlc_df["Close"] * 100, mode="lines", name=f"{curr.date()}"))
    except Exception as e:
        print(f"{curr} NOT IN DB", e)

fig.update_layout(
    title="SOFR OIS NY Close Curve", xaxis_title="Tenor", yaxis_title="Fixed Rate", template="plotly_dark", height=1000, showlegend=True
)
fig.update_xaxes(
    showgrid=True,
    showspikes=True,
    spikecolor="white",
    spikesnap="cursor",
    spikemode="across",
)
fig.update_yaxes(showgrid=True, showspikes=True, spikecolor="white", spikesnap="cursor", spikethickness=0.5)
fig.show()

In [26]:
start_date = datetime(2018, 1, 1)
end_date = datetime(2024, 11, 26)
sofr_ois_df = bbg_490_curve.s490_nyclose_term_structure_ts(start_date=start_date, end_date=end_date)
ct_df = curve_data_fetcher.fedinvest_data_fetcher.get_historical_ct_yields(start_date=start_date, end_date=end_date)

In [27]:
swap_tenors = ["2Y", "5Y", "10Y", "30Y", "50Y"]
ct_tenors = ["CT10"]

fig = go.Figure()
for tenor in swap_tenors:
    fig.add_trace(go.Scatter(x=sofr_ois_df["Date"], y=sofr_ois_df[tenor], mode="lines", name=f"{tenor} Swap"))

for tenor in ct_tenors:    
    fig.add_trace(go.Scatter(x=ct_df["Date"], y=ct_df[tenor], mode="lines", name=f"{tenor}"))

fig.update_layout(title=f"Levels: SOFR OIS, CT Yields", xaxis_title="Date", yaxis_title="Fixed Rate", template="plotly_dark", height=1000)
fig.update_xaxes(
    showgrid=True,
    showspikes=True,
    spikecolor="white",
    spikesnap="cursor",
    spikemode="across",
)
fig.update_yaxes(showgrid=True, showspikes=True, spikecolor="white", spikesnap="cursor", spikethickness=0.5)
fig.show()

In [28]:
swap_curve_spreads = [("2Y", "10Y"), ("2Y", "5Y"), ("5Y", "30Y")]
ct_curve_spreads = [("CT2", "CT10")]

fig = go.Figure()
for spread in swap_curve_spreads:
    fig.add_trace(go.Scatter(x=sofr_ois_df["Date"], y=(sofr_ois_df[spread[1]] - sofr_ois_df[spread[0]]) * 100, mode="lines", name=f"{spread[0]}s{spread[1]}s"))

for spread in ct_curve_spreads:
    fig.add_trace(go.Scatter(x=ct_df["Date"], y=(ct_df[spread[1]] - ct_df[spread[0]]) * 100, mode="lines", name=f"{spread[0]}s{spread[1]}s"))

fig.update_layout(title=f"Slopes: SOFR OIS, CT Yields", xaxis_title="Date", yaxis_title="Fixed Rate", template="plotly_dark", height=1000)
fig.update_xaxes(
    showgrid=True,
    showspikes=True,
    spikecolor="white",
    spikesnap="cursor",
    spikemode="across",
)
fig.update_yaxes(showgrid=True, showspikes=True, spikecolor="white", spikesnap="cursor", spikethickness=0.5)
fig.show()

In [29]:
swap_curve_flies = [("2Y", "5Y", "10Y")]
ct_curve_flies = [("CT2", "CT5", "CT10")]

fig = go.Figure()

for fly in swap_curve_flies:
    fig.add_trace(
        go.Scatter(
            x=sofr_ois_df["Date"],
            y=((sofr_ois_df[fly[1]] - sofr_ois_df[fly[0]]) - (sofr_ois_df[fly[2]] - sofr_ois_df[fly[1]])) * 100,
            mode="lines",
            name=f"{fly[0]}s{fly[1]}s{fly[2]}s",
        )
    )

for fly in ct_curve_flies:
    fig.add_trace(
        go.Scatter(
            x=ct_df["Date"],
            y=((ct_df[fly[1]] - ct_df[fly[0]]) - (ct_df[fly[2]] - ct_df[fly[1]])) * 100,
            mode="lines",
            name=f"{fly[0]}s{fly[1]}s{fly[2]}s",
        )
    )

fig.update_layout(title=f"Curvature: SOFR OIS, CT Yields", xaxis_title="Date", yaxis_title="Fixed Rate", template="plotly_dark", height=1000, showlegend=True)
fig.update_xaxes(
    showgrid=True,
    showspikes=True,
    spikecolor="white",
    spikesnap="cursor",
    spikemode="across",
)
fig.update_yaxes(showgrid=True, showspikes=True, spikecolor="white", spikesnap="cursor", spikethickness=0.5)
fig.show()

# SOFR OIS Swap Spreads:

In [34]:
ct_df = curve_data_fetcher.fedinvest_data_fetcher.get_historical_ct_yields(start_date=start_date, end_date=end_date)
cmt_df = curve_data_fetcher.fred_data_fetcher.get_historical_cmt_yields(start_date=start_date, end_date=end_date)

swap_spread_ct_df = bbg_490_curve.swap_spreads_term_structure(
    swaps_term_structure_ts_df=sofr_ois_df.set_index("Date"), cash_term_structure_ts_df=ct_df.set_index("Date")
)

swap_spread_cmt_df = bbg_490_curve.swap_spreads_term_structure(
    swaps_term_structure_ts_df=sofr_ois_df.set_index("Date"), cash_term_structure_ts_df=cmt_df.set_index("Date"), is_cmt=True
)

# swap_spread_ct_df
swap_spread_cmt_df

Fetching from FRED...


,3M,6M,12M,2Y,3Y,5Y,7Y,10Y,20Y,30Y
Date,,,,,,,,,,
2018-01-02,5.740000,-0.680000,-10.790000,-3.520000,-5.990000,-25.600000,-34.720000,-37.730000,-47.030000,-64.780000
2018-01-03,9.220000,2.320000,-7.400000,-4.580000,-6.390000,-25.200000,-34.040000,-36.700000,-46.430000,-63.220000
2018-01-04,10.450000,2.910000,-6.890000,-4.180000,-6.590000,-24.200000,-32.660000,-37.030000,-45.960000,-63.610000
2018-01-05,12.450000,4.810000,-4.570000,-3.480000,-6.490000,-24.400000,-32.820000,-35.990000,-45.380000,-63.120000
2018-01-08,6.580000,2.550000,-4.120000,-4.320000,-8.590000,-25.200000,-34.230000,-38.280000,-46.370000,-62.960000
...,...,...,...,...,...,...,...,...,...,...
2024-11-19,12.498008,11.846445,-6.090985,-19.836313,-25.711887,-34.970525,-44.169548,-50.516938,-75.935394,-83.873893
2024-11-20,11.828305,12.931377,-7.143563,-20.485696,-24.364376,-34.957952,-42.925988,-49.372456,-72.936349,-82.863330
2024-11-21,10.251742,11.699638,-7.325407,-20.505568,-25.198415,-34.318675,-42.730367,-49.238906,-72.562576,-82.802169


In [36]:
as_of_dates = [datetime(2019, 1, 3), datetime(2020, 1, 3), datetime(2021, 1, 3), datetime(2022, 1, 3), datetime(2023, 1, 4), datetime(2024, 1, 2), datetime(2024, 11, 25)]

fig = go.Figure()
for curr in as_of_dates:
    try:
        spreads_dict = swap_spread_cmt_df[swap_spread_cmt_df.index == curr].to_dict("records")[0]
        fig.add_trace(
            go.Scatter(x=[tenor_to_years(tenor) for tenor in spreads_dict.keys()], y=list(spreads_dict.values()), mode="lines", name=f"{curr.date()}")
        )
    except Exception as e:
        print(f"{curr} NOT IN DB", e)

fig.update_layout(title="SOFR OIS Swap Spreads NY Close Curve", xaxis_title="Tenor", yaxis_title="Fixed Rate", template="plotly_dark", height=1000)
fig.update_xaxes(
    showgrid=True,
    showspikes=True,
    spikecolor="white",
    spikesnap="cursor",
    spikemode="across",
)
fig.update_yaxes(showgrid=True, showspikes=True, spikecolor="white", spikesnap="cursor", spikethickness=0.5)
fig.show()

2021-01-03 00:00:00 NOT IN DB list index out of range


In [ ]:
tenors = ["5Y", "10Y"]
fig = go.Figure()
for tenor in tenors:
    fig.add_trace(go.Scatter(x=swap_spread_cmt_df.index, y=swap_spread_cmt_df[tenor], mode="lines", name=f"{tenor}"))

fig.update_layout(
    title="SOFR OIS Swap Spreads NY Close Curve", xaxis_title="Tenor", yaxis_title="Fixed Rate", template="plotly_dark", height=1000, showlegend=True
)
fig.update_xaxes(
    showgrid=True,
    showspikes=True,
    spikecolor="white",
    spikesnap="cursor",
    spikemode="across",
)
fig.update_yaxes(showgrid=True, showspikes=True, spikecolor="white", spikesnap="cursor", spikethickness=0.5)
fig.show()

In [40]:
swap_spread_slopes = [("5Y", "10Y")]
fig = go.Figure()
for slope in swap_spread_slopes:
    fig.add_trace(
        go.Scatter(
            x=swap_spread_cmt_df.index,
            y=(swap_spread_cmt_df[slope[1]] - swap_spread_cmt_df[slope[0]]) * 100,
            mode="lines",
            name=f"{slope[0]}s{slope[1]}s",
        )
    )

fig.update_layout(
    title="SOFR OIS Swap Spreads Slope NY Close Curve", xaxis_title="Tenor", yaxis_title="Fixed Rate", template="plotly_dark", height=1000, showlegend=True
)
fig.update_xaxes(
    showgrid=True,
    showspikes=True,
    spikecolor="white",
    spikesnap="cursor",
    spikemode="across",
)
fig.update_yaxes(showgrid=True, showspikes=True, spikecolor="white", spikesnap="cursor", spikethickness=0.5)
fig.show()

# SOFR OIS Forwards

In [42]:
start_date = datetime(2024, 11, 1)
end_date = datetime(2024, 11, 27)

fwd_dict, _ = bbg_490_curve.s490_nyclose_fwd_curve_matrices(
    start_date=start_date,
    end_date=end_date,
    ql_piecewise_method="logLinearDiscount",
    # ql_zero_curve_method=ql.LogLinearZeroCurve,
    ql_compounding=ql.Simple,
    # bp_fixed_adjustment=-10,
)

In [43]:
fwd_tenors = ["12M Fwd 10Y"]

fig = go.Figure()

dates = []
fwds: Dict[str, List[float]] = {}
for bdate in pd.date_range(start=start_date, end=end_date, freq=CustomBusinessDay(calendar=USFederalHolidayCalendar())):
    try:
        for fwd_tenor in fwd_tenors:
            if fwd_tenor not in fwds:
                fwds[fwd_tenor] = []
            groups = fwd_tenor.split(" ")
            option_tenor = " ".join(groups[:2])
            swap_tenor = " ".join(groups[2:])
            fwds[fwd_tenor].append(fwd_dict[bdate.to_pydatetime()].loc[fwd_dict[bdate.to_pydatetime()]["Tenor"] == swap_tenor, option_tenor].values[0])
        dates.append(bdate.to_pydatetime())
    except Exception as e:
        print(f"{bdate} not in db", e)
        # pass

for fwd_tenor, fwd_rates in fwds.items():
    fig.add_trace(
        go.Scatter(
            x=dates,
            y=fwd_rates,
            mode="lines",
            name=fwd_tenor,
        )
    )

fig.update_layout(title="SOFR OIS Swap Fwd Rates", xaxis_title="Date", yaxis_title="Fixed Rate", template="plotly_dark", height=1000, showlegend=True)
fig.update_xaxes(
    showgrid=True,
    showspikes=True,
    spikecolor="white",
    spikesnap="cursor",
    spikemode="across",
)
fig.update_yaxes(showgrid=True, showspikes=True, spikecolor="white", spikesnap="cursor", spikethickness=0.5)
fig.show()

In [44]:
fig = go.Figure()
fwd_tenor = "5Y Fwd"
fwd_curve_date = datetime(2024, 11, 26)
display(fwd_dict[fwd_curve_date])

fig.add_trace(
    go.Scatter(
        x=[tenor_to_years(tenor) for tenor in fwd_dict[fwd_curve_date]["Tenor"].to_list()],
        y=fwd_dict[fwd_curve_date][fwd_tenor],
        mode="lines",
        name=fwd_tenor,
    )
)
fig.update_layout(
    title="SOFR OIS Swap Fwd Curve", xaxis_title="Tenor", yaxis_title="Fixed Rate", template="plotly_dark", height=1000, showlegend=True
)
fig.update_xaxes(
    showgrid=True,
    showspikes=True,
    spikecolor="white",
    spikesnap="cursor",
    spikemode="across",
)
fig.update_yaxes(showgrid=True, showspikes=True, spikecolor="white", spikesnap="cursor", spikethickness=0.5)
fig.show()

,Tenor,Fixed Rate,1D Fwd,1W Fwd,1M Fwd,2M Fwd,3M Fwd,4M Fwd,5M Fwd,6M Fwd,...,12M Fwd,18M Fwd,2Y Fwd,3Y Fwd,4Y Fwd,5Y Fwd,7Y Fwd,10Y Fwd,15Y Fwd,20Y Fwd
0,1D,4.580000,4.897918,4.900000,4.544774,4.401742,4.418192,4.430347,4.330262,4.330388,...,4.190581,4.066827,4.060730,4.040811,4.048458,4.081878,4.201916,4.479819,5.048188,5.569324
1,1W,4.609523,4.897918,4.900000,4.274993,4.354739,4.384770,4.390726,4.295733,4.304022,...,4.171069,4.052320,4.049929,4.032640,4.041918,4.076483,4.197933,4.477015,5.045837,5.566571
2,2W,4.596349,4.897918,4.900000,4.164193,4.299123,4.346323,4.326100,4.267135,4.265565,...,4.145392,4.036863,4.036545,4.022843,4.034244,4.070345,4.193493,4.473811,5.042852,5.562851
3,3W,4.583212,4.897918,4.124098,4.053403,4.243513,4.307879,4.261487,4.238540,4.227115,...,4.119723,4.021410,4.023164,4.013049,4.026572,4.064208,4.189055,4.470608,5.039867,5.559131
4,1M,4.565755,3.088897,3.844523,4.289239,4.359194,4.389714,4.266139,4.278990,4.237790,...,4.115563,4.021726,4.020934,4.010671,4.024635,4.062964,4.188543,4.470649,5.039297,5.557011
5,2M,4.531655,4.408985,4.410679,4.417466,4.425401,4.255514,4.261840,4.217533,4.181314,...,4.061062,3.992937,3.992436,3.988745,4.007353,4.049578,4.179417,4.464650,5.032837,5.546945
6,3M,4.497809,4.408985,4.410679,4.416617,4.153873,4.190942,4.149354,4.116108,4.095717,...,3.990756,3.953909,3.955714,3.960978,3.985514,4.032473,4.167508,4.456544,5.024560,5.534953
7,4M,4.471935,4.408985,4.235741,3.902496,4.072695,4.053422,4.032621,4.022508,4.016228,...,3.927296,3.919107,3.922568,3.935958,3.965802,4.017034,4.157006,4.449228,5.017352,5.524133
8,5M,4.446210,3.797155,4.010669,4.233630,4.115592,4.062602,4.038739,4.025054,4.018129,...,3.902417,3.910901,3.909448,3.924541,3.956671,4.010477,4.153213,4.447494,5.014248,5.516825
9,6M,4.420633,4.292074,4.187178,3.986604,3.963814,3.960523,3.959522,3.961207,3.964701,...,3.850115,3.884128,3.882087,3.903353,3.939943,3.997579,4.144500,4.441854,5.008031,5.506858
